In [21]:
from joblib import Parallel, delayed
import multiprocessing
import pandas as pd
import random
from textblob import TextBlob
from textblob.sentiments import NaiveBayesAnalyzer

In [22]:
random.seed(20191024)

In [23]:
sentiment140 = pd.read_csv('../data/sentiment_140_clean.csv', encoding='ISO-8859-1')
sentiment140.head()

,sentiment,text
0,0,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,is upset that he can't update his Facebook by ...
2,0,@Kenichan I dived many times for the ball. Man...
3,0,my whole body feels itchy and like its on fire
4,0,"@nationwideclass no, it's not behaving at all...."


In [ ]:
# Sample 10k records for model test set
sample = sentiment140.sample(10000)

In [36]:
%%time
# Analyze sentiments in parallel
def process_sentiment(tweet):
    return TextBlob(tweet, analyzer=NaiveBayesAnalyzer()).sentiment[0]

n_cores = multiprocessing.cpu_count()
sentiments = Parallel(n_jobs=n_cores, verbose=2)(delayed(process_sentiment)(tweet) for i in sample['text'])

[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  25 tasks      | elapsed:   38.3s
[Parallel(n_jobs=8)]: Done 146 tasks      | elapsed:  3.2min


KeyboardInterrupt: 